## Keras layers


In [1]:
import tensorflow as tf
import tensorflow.keras.backend as K
from collections import Counter


# Class representing AGDS
class SparseLayer(tf.keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        super(SparseLayer, self).__init__(**kwargs)
        self.output_dim = output_dim
        

    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[1], self.output_dim), 
            initializer='random_normal',
            trainable=True)
        super(SparseLayer, self).build(input_shape)

    def call(self, input_data):
        multi_kernel = K.expand_dims(self.kernel, 0)
        coeffs = tf.keras.layers.Dot(axes=(1, 1))([input_data, multi_kernel])
        return tf.keras.activations.softmax(coeffs)
    
    def get_config(self):
        return {"output_dimension": self.output_dim}


# Class for AGDS Word2BoW processing
class AGDSVectorization(tf.keras.layers.Layer):
    def __init__(self, word_map, **kwargs):
        self.word_map = word_map
        super(AGDSVectorization, self).__init__(**kwargs)

    def call(self, input_data):
        post_vector = [0] * len(self.word_map)

        # Calculate word occurrences
        word_ctr = Counter(input_data)

        for word, freq in word_ctr.items():
            if word in self.word_map:
                post_vector[self.word_map.index(word)] = freq
        return post_vector

    def compute_output_shape(self, input_shape):
        return (input_shape[0], len(self.word_map))

## Model testing

In [2]:
# Associate words with archetypes/character traits as intermediate layer
# and with influencer as the "last" layer

# Dependencies
import pandas as pd
import numpy as np
from tqdm.notebook import trange, tqdm
import copy
import os
import toml
import re
import itertools
from text_cleaner import *
import operator
from collections import Counter
import pickle

def extract_hashtags(post_text):
    HASH_RE = re.compile(r"\#\w+")
    out_list = re.findall(HASH_RE, post_text)
    return out_list

In [3]:
# Load the .csv with archetypes
arch_df = pd.read_csv('archetypes_pl_new.csv', index_col=0)

# Save the order of columns
trait_list = arch_df.columns.tolist()

# Show the table header and column list
print(trait_list)
arch_df.head()

['innocent', 'sage', 'explorer', 'outlaw', 'magician', 'hero', 'lover', 'jester', 'everyman', 'caregiver', 'ruler', 'creator', 'dominant', 'submissive', 'maximalist', 'minimalist', 'inspiring', 'systematic', 'discovering', 'conservative', 'verifying', 'overlooking', 'sharpening', 'harmonic', 'empathic', 'matter_of_fact', 'brave', 'protective', 'generous', 'thrifty', 'favourable', 'balanced', 'sensuality', 'intelligent', 'believe', 'egocentric', 'allocentric']


,innocent,sage,explorer,outlaw,magician,hero,lover,jester,everyman,caregiver,...,protective,generous,thrifty,favourable,balanced,sensuality,intelligent,believe,egocentric,allocentric
id,,,,,,,,,,,,,,,,,,,,,
marek_grodzki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,2.0,3.0,4.0,4.0,3.0,4.0,4.0,3.0,0.0,0.0
vege_style_life,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,0.0,3.0
oliwka__2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,2.0,2.0,0.0,3.0,1.0,2.0,4.0,1.0,0.0,3.0
z_przestrzeni_serca,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0.0,4.0,4.0,3.0,4.0,4.0,0.0,1.0
zaradne_warsztaty,3.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,3.0,4.0,...,3.0,4.0,0.0,2.0,2.0,4.0,2.0,3.0,1.0,3.0


In [4]:
# Load data
train_df = pd.read_csv("agds_structures/train_90.csv", index_col=0)
test_df = pd.read_csv("agds_structures/test_10.csv", index_col=0)

with open("agds_structures/vectorized_test_90.pickle", "rb") as f:
    test_vectorized = pickle.load(f)
    
with open("agds_structures/vectorized_train_90.pickle", "rb") as f:
    train_vectorized = pickle.load(f)
    
# Load structure
with open("agds_structures/normalized_s90_10_word_trait_array.pickle", "rb") as f:
    softmax_word_df = pickle.load(f)

# Extract word map
softmax_word_map = softmax_word_df.columns.tolist()
    
# Show array head
softmax_word_df.head()

good   morning      wish       you      nice  relaxing  \
innocent 0  1.000000  1.000000  1.000000  0.985589  1.000000  1.000000   
         1  0.164918  0.363648  0.005689  1.000000  0.000000  0.000000   
         2  0.059452  0.032827  0.000000  0.885980  0.026943  0.003168   
         3  0.053402  0.063776  0.003512  0.250463  0.024454  0.000000   
         4  0.000000  0.000000  0.000000  0.000000  0.013889  0.000000   

            thursday  saturday    friday     start  ...  #używane  \
innocent 0  1.000000  1.000000  0.579753  1.000000  ...       0.0   
         1  0.000000  0.000000  1.000000  0.437572  ...       0.0   
         2  0.000000  0.000000  0.097603  0.325299  ...       0.0   
         3  0.021592  0.062854  0.216949  0.151777  ...       0.0   
         4  0.000000  0.000000  0.000000  0.000000  ...       0.0   

            #podzielnia  #ekobiuro  #goHi2020  #hackaton  #jachranka  \
innocent 0          0.0        0.0        0.0        0.0         0.0   
         1          0.0        0.0        0.0        0.0         0.0   
         2          0.0        0.0        0.0        0.0         0.0   
         3          0.0        0.0        0.0        0.0         0.0   
         4          0.0        0.0        0.0        0.0         0.0   

            #greenladies  #greenguys  #polishheroes  #bestgifts  
innocent 0           0.0         0.0            0.0         0.0  
         1           0.0         0.0            0.0         0.0  
         2           0.0         0.0            0.0         0.0  
         3           0.0         0.0            0.0         0.0  
         4           0.0         0.0            0.0         0.0  

[5 rows x 211596 columns]

In [5]:
# Prepare training data
X_train = np.array(train_vectorized)
print(len(X_train))

# Prepare validation/test dataset
X_val = np.array(test_vectorized)

616


In [7]:
# Create a model
def build_model():
    inputs = tf.keras.Input(shape=(len(softmax_word_map), ), name="input_layer")
    x = SparseLayer(5, name="AGDS_weight_layer")(inputs)
    outputs = tf.keras.activations.softmax(x)

    model = tf.keras.Model(
        inputs=inputs,
        outputs=[outputs]
    )
    return model

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs_agds_model")
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,
                              patience=5, min_lr=1e-7, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', min_delta=0, patience=20, verbose=1,
    mode='auto', baseline=None, restore_best_weights=True)

In [8]:
def trait_training_pipeline(trait_list):
    for trait in tqdm(trait_list):
        # Prepare labels
        y_train = tf.keras.utils.to_categorical(np.array(train_df[trait]), num_classes=5)
        y_val = tf.keras.utils.to_categorical(np.array(test_df[trait]), num_classes=5)

        # Create model
        test_model = build_model()

        test_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-2),
                       loss=tf.keras.losses.CategoricalCrossentropy(),
                       metrics=["accuracy"])

        # Set callbacks
        model_save_callback = tf.keras.callbacks.ModelCheckpoint("./agds_structures/weight_finetuning/"+trait+"-saved-model-{epoch:02d}-{val_accuracy:.2f}.hdf5", 
                                                              monitor='val_accuracy', 
                                                              verbose=1, save_best_only=True, save_weights_only=True, 
                                                              mode='max')
        

        # Train the model
        with tf.device("/GPU:0"):
            out = test_model.fit(X_train, 
                                 y_train,
                                 batch_size=10,
                                 validation_data=(X_val, y_val),
                                 epochs=100,
                                 callbacks=[model_save_callback, tensorboard_callback, reduce_lr, early_stop])

        weights_tmp = test_model.get_layer("AGDS_weight_layer").get_weights()[0]
        softmax_word_df.loc[trait] = weights_tmp.T
        
        del y_train
        del y_val
        del test_model
        del out
        del weights_tmp

In [9]:
trait_training_pipeline(trait_list)

  0%|          | 0/37 [00:00<?, ?it/s]

Epoch 1/100
62/62 [==============================] - 3s 18ms/step - loss: 1.3657 - accuracy: 0.5373 - val_loss: 1.2835 - val_accuracy: 0.6232

Epoch 00001: val_accuracy improved from -inf to 0.62319, saving model to ./agds_structures/weight_finetuning/innocent-saved-model-01-0.62.hdf5
Epoch 2/100
62/62 [==============================] - 1s 9ms/step - loss: 1.2753 - accuracy: 0.6266 - val_loss: 1.3065 - val_accuracy: 0.5942

Epoch 00002: val_accuracy did not improve from 0.62319
Epoch 3/100
62/62 [==============================] - 1s 9ms/step - loss: 1.2628 - accuracy: 0.6429 - val_loss: 1.3128 - val_accuracy: 0.5942

Epoch 00003: val_accuracy did not improve from 0.62319
Epoch 4/100
62/62 [==============================] - 1s 9ms/step - loss: 1.2181 - accuracy: 0.6851 - val_loss: 1.3344 - val_accuracy: 0.5652

Epoch 00004: val_accuracy did not improve from 0.62319
Epoch 5/100
62/62 [==============================] - 1s 9ms/step - loss: 1.2075 - accuracy: 0.6964 - val_loss: 1.3112 - val

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/mnt/HDD_Linux/Praca_magisterska/instagram_analysis/insta_venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-8b4b62bc367b>", line 1, in <module>
    trait_training_pipeline(trait_list)
  File "<ipython-input-8-17c622f02d43>", line 23, in trait_training_pipeline
    out = test_model.fit(X_train,
  File "/mnt/HDD_Linux/Praca_magisterska/instagram_analysis/insta_venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1183, in fit
    tmp_logs = self.train_function(iterator)
  File "/mnt/HDD_Linux/Praca_magisterska/instagram_analysis/insta_venv/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 889, in __call__
    result = self._call(*args, **kwds)
  File "/mnt/HDD_Linux/Praca_magisterska/instagram_analysis/insta_venv/lib/python3.8/site-packages/tensorflow/python/eager/def_functi

TypeError: object of type 'NoneType' has no len()

# Test stuff

In [17]:
# Prepare weights for 

test_model.get_layer("AGDS_weight_layer").set_weights([softmax_word_df.loc["innocent"].to_numpy().T])

In [18]:
test_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-2),
                   loss=tf.keras.losses.CategoricalCrossentropy(),
                   metrics=["accuracy"])

# Set callbacks
model_save_callback = tf.keras.callbacks.ModelCheckpoint("./agds_structures/weight_finetuning/saved-model-{epoch:02d}-{val_accuracy:.2f}.hdf5", 
                                                      monitor='val_accuracy', 
                                                      verbose=1, save_best_only=True, save_weights_only=True, 
                                                      mode='auto')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs_agds_model")
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,
                              patience=5, min_lr=1e-7, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', min_delta=0, patience=20, verbose=1,
    mode='auto', baseline=None, restore_best_weights=True)

with tf.device("/GPU:0"):
    out = test_model.fit(X_train, 
                         y_train,
                         batch_size=10,
                         validation_data=(X_val, y_val),
                         epochs=100,
                         callbacks=[model_save_callback, tensorboard_callback, reduce_lr, early_stop])

Epoch 1/100
62/62 [==============================] - 1s 14ms/step - loss: 1.1431 - accuracy: 0.7614 - val_loss: 1.0792 - val_accuracy: 0.8261

Epoch 00001: val_accuracy improved from -inf to 0.82609, saving model to ./agds_structures/saved-model-01-0.83.hdf5
Epoch 2/100
62/62 [==============================] - 1s 9ms/step - loss: 1.0897 - accuracy: 0.8149 - val_loss: 1.0637 - val_accuracy: 0.8406

Epoch 00002: val_accuracy improved from 0.82609 to 0.84058, saving model to ./agds_structures/saved-model-02-0.84.hdf5
Epoch 3/100
62/62 [==============================] - 1s 9ms/step - loss: 1.0501 - accuracy: 0.8539 - val_loss: 1.0280 - val_accuracy: 0.8696

Epoch 00003: val_accuracy improved from 0.84058 to 0.86957, saving model to ./agds_structures/saved-model-03-0.87.hdf5
Epoch 4/100
62/62 [==============================] - 1s 9ms/step - loss: 1.0146 - accuracy: 0.8896 - val_loss: 1.0208 - val_accuracy: 0.8841

Epoch 00004: val_accuracy improved from 0.86957 to 0.88406, saving model to .

In [38]:
test_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 211596)]          0         
_________________________________________________________________
AGDS_weight_layer (SparseLay (None, 5)                 1057980   
_________________________________________________________________
tf.compat.v1.nn.softmax (TFO (None, 5)                 0         
Total params: 1,057,980
Trainable params: 1,057,980
Non-trainable params: 0
_________________________________________________________________


In [19]:
weights_tmp = test_model.get_layer("AGDS_weight_layer").get_weights()[0]
print(weights_tmp)
print()

[[ 9.6871972e-01  7.6641291e-02 -1.3664700e-01  3.3654189e-01
  -1.1755135e-01]
 [ 1.0776323e+00  3.0333450e-01 -3.6403559e-02  2.3373212e-01
  -9.4700076e-02]
 [ 1.0000000e+00  3.1873159e-02 -9.4676167e-02  6.7399435e-02
  -3.1427972e-02]
 ...
 [ 0.0000000e+00  4.4363305e-02 -4.4368908e-02  1.8069198e-18
   0.0000000e+00]
 [ 0.0000000e+00  4.4363305e-02 -4.4368908e-02  1.8069198e-18
   0.0000000e+00]
 [ 0.0000000e+00  4.4363305e-02 -4.4368908e-02  1.8069198e-18
   0.0000000e+00]]



In [20]:
# Try to set the dataframe with new weights
softmax_word_df.loc["innocent"] = weights_tmp.T

In [21]:
softmax_word_df.head()

good   morning      wish       you      nice      relaxing  \
innocent 0  0.968720  1.077632  1.000000  0.922863  1.000000  1.000000e+00   
         1  0.076641  0.303335  0.031873  0.898432 -0.014613  0.000000e+00   
         2 -0.136647 -0.036404 -0.094676  0.711773  0.089594  3.168382e-03   
         3  0.336542  0.233732  0.067399  0.597796  0.042476  1.589736e-10   
         4 -0.117551 -0.094700 -0.031428 -0.173881 -0.017539  0.000000e+00   

                thursday      saturday        friday     start  ...  \
innocent 0  1.000000e+00  1.000000e+00  5.797529e-01  0.978891  ...   
         1 -2.934082e-31 -1.018130e-01  9.946581e-01  0.350028  ...   
         2  0.000000e+00 -2.369519e-02  3.892035e-02  0.199146  ...   
         3  2.159156e-02  1.882142e-01  2.493641e-01  0.341023  ...   
         4  0.000000e+00 -8.486718e-30 -1.334374e-22 -0.012682  ...   

                #używane   #podzielnia     #ekobiuro     #goHi2020  \
innocent 0  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
         1  4.436330e-02  4.436330e-02  4.436330e-02  4.436330e-02   
         2 -4.436891e-02 -4.436891e-02 -4.436891e-02 -4.436891e-02   
         3  1.806920e-18  1.806920e-18  1.806920e-18  1.806920e-18   
         4  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   

               #hackaton    #jachranka  #greenladies    #greenguys  \
innocent 0  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
         1  4.436330e-02  4.436330e-02  4.436330e-02  4.436330e-02   
         2 -4.436891e-02 -4.436891e-02 -4.436891e-02 -4.436891e-02   
         3  1.806920e-18  1.806920e-18  1.806920e-18  1.806920e-18   
         4  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   

            #polishheroes    #bestgifts  
innocent 0   0.000000e+00  0.000000e+00  
         1   4.436330e-02  4.436330e-02  
         2  -4.436891e-02 -4.436891e-02  
         3   1.806920e-18  1.806920e-18  
         4   0.000000e+00  0.000000e+00  

[5 rows x 211596 columns]